## 주식 공시 분석


### 다음은 네이버 주식 공시를 분석하는 파이썬 알고리즘 입니다.


뉴스 제목과 기사 링크를 리스트에 담는 알고리즘

In [1]:
import requests as rq
import re
import datetime

URL = "https://finance.naver.com"
news_list = "/news/news_list.nhn"

today = str(datetime.date.today())
today = today.strip().split("-")
date = ""

time_warp = int(input('몇일 전 공시를 보시겠습니까?: '))
today[2] = int(today[2]) - time_warp

if today[2] < 10:
    today[2] = '0' + str(today[2])

for i in range(3):
    date += str(today[i])

subject_list = []
read_link_list = []

pattern_string1 = r'<!-- 상단 10개의 기사 -->'
pattern1 = re.compile(pattern_string1)
pattern_string2 = r'<!-- 하단 10개의 기사 끝 -->'
pattern2 = re.compile(pattern_string2)

pattern_link = r'<a href="/news/news_read.nhn?'
pattern3 = re.compile(pattern_link)

pattern_title = r'title="'
pattern4 = re.compile(pattern_title)
pattern_title_end = r'">'
pattern5 = re.compile(pattern_title_end)

for i in range(1, 11):
    page = i
    parameter = {"mode": "LSS3D", "section_id": 101, "section_id2": 258, "section_id3": 406, "date": date, 'page': page}

    res = rq.post(URL + news_list, params=parameter)
    res_code = res.status_code

    if res_code == 200:
        print("OKAY")
        res_text = res.text
        # print(res_text)
        match_check1 = re.search(pattern1, res_text)
        match_check2 = re.search(pattern2, res_text)
        if match_check1 != None and match_check2 != None:
            rang1 = match_check1.span()
            rang2 = match_check2.span()
            
            news_text = res_text[rang1[1]:rang2[0]]
            # print(news_text)
            
            start_idx = 0
            
            for i in range(10):
                
                try:
                    news_text = news_text[start_idx:]
                    # print(i)
                    match_link = re.search(pattern3, news_text)
                    rang_link = match_link.span()[0]
                    link_text = news_text[rang_link+9:rang_link+150]
                    # print(link_text)
            
                    match_title = re.search(pattern4, news_text)
                    rang_title = match_title.span()[0]
                    title_text = news_text[rang_title+7:rang_title+150]
                    
                    match_title_end = re.search(pattern5, title_text)
                    rang_title_end = match_title_end.span()[0]
                    title_text = title_text[:rang_title_end]
                    # print(title_text)
                
                    start_idx = rang_title + 150 + rang_title_end
                    # print("시작 인덱스: ", start_idx)
                    
                    subject_list.append(title_text)
                    read_link_list.append(link_text)
                    
                except AttributeError:
                    pass
            
        res_data = res.url
        print(res_data)
    else:
        print(res_code)
        print("FAIL")

몇일 전 공시를 보시겠습니까?: 1
OKAY
https://finance.naver.com/news/news_list.nhn?section_id2=258&section_id=101&date=20190809&mode=LSS3D&page=1&section_id3=406
OKAY
https://finance.naver.com/news/news_list.nhn?section_id2=258&section_id=101&date=20190809&mode=LSS3D&page=2&section_id3=406
OKAY
https://finance.naver.com/news/news_list.nhn?section_id2=258&section_id=101&date=20190809&mode=LSS3D&page=3&section_id3=406
OKAY
https://finance.naver.com/news/news_list.nhn?section_id2=258&section_id=101&date=20190809&mode=LSS3D&page=4&section_id3=406
OKAY
https://finance.naver.com/news/news_list.nhn?section_id2=258&section_id=101&date=20190809&mode=LSS3D&page=5&section_id3=406
OKAY
https://finance.naver.com/news/news_list.nhn?section_id2=258&section_id=101&date=20190809&mode=LSS3D&page=6&section_id3=406
OKAY
https://finance.naver.com/news/news_list.nhn?section_id2=258&section_id=101&date=20190809&mode=LSS3D&page=7&section_id3=406
OKAY
https://finance.naver.com/news/news_list.nhn?section_id2=258&section_id=

제목을 기반으로 호재인 기사만 추출

In [4]:
good_news_subject = []
good_news_link = []

good_judge_list = ['배당', '수주', '&uarr', '증가', '기대', '확대', '체결', '올라']

idx = 0
while(idx < len(subject_list)):
    check = 0
    for item in good_judge_list:
        if idx < len(subject_list) and item in subject_list[idx]:
            good_news_subject.append(subject_list[idx])
            good_news_link.append(read_link_list[idx])
            idx += 1
            check += 1
    if check == 0:
        idx += 1

호재인 기사 출력

In [5]:
for i in range(len(good_news_subject)):
    res_good_news = rq.post(URL + good_news_link[i])
    print(good_news_subject[i])
    print(res_good_news.url)

지투알, 올 2Q 영업익 41억원&hellip;전년比 584.9%&uarr;
https://finance.naver.com/news/news_read.nhn?article_id=0004442212&office_id=018&mode=LSS3D&type=0&section_id=101&section_id2=258&section_id3=406&date=20190809&page=2
인터파크홀딩스, 2분기 영업이익 126억원&hellip;전년비 184%&uarr;
https://finance.naver.com/news/news_read.nhn?article_id=0004442199&office_id=018&mode=LSS3D&type=0&section_id=101&section_id2=258&section_id3=406&date=20190809&page=2
CS홀딩스, 올 2Q 영업익 46억원&hellip;전년比 45.5%&uarr;
https://finance.naver.com/news/news_read.nhn?article_id=0004442198&office_id=018&mode=LSS3D&type=0&section_id=101&section_id2=258&section_id3=406&date=20190809&page=2
지투알, 2분기 영업익 41억원&hellip;584.9% 증가
https://finance.naver.com/news/news_read.nhn?article_id=0004516523&office_id=277&mode=LSS3D&type=0&section_id=101&section_id2=258&section_id3=406&date=20190809&page=2
현대홈쇼핑, 2Q 연결 잠정 영업익 454억원&hellip;전년比 42%&uarr;
https://finance.naver.com/news/news_read.nhn?article_id=0004516507&office_id=277&mode=LSS3D&type=0&section_id=101&sec

호재인 종목명 리스트에 담고 .txt파일로 내보내기

In [10]:
code_name = []
text = ''

for item in good_news_subject:
    name = item.split(' ')[0].strip(',').strip('[공시+]')
    print(name)
    code_name.append(name)
    text += name + ','
    
f = open('C:/VISION_FINAL_0.4/종목추천/' + str(date) + ' 공시 기반 종목 추천.txt', 'wt', encoding='utf-8')
f.write(text)
f.close()

지투알
인터파크홀딩스
CS홀딩스
지투알
현대홈쇼핑
오스템임플란트
인터파크홀딩스
유니테스트
정상제이엘에스
상상인
신라젠
상상인
오스템임플란트
신세계푸드
제노레이
유니트론텍
롯데쇼핑
코오롱인더
코오롱인더
데브시스터즈
데브시스터즈
신세계푸드
제노레이
신라젠
유진테크
게임빌
NHN한국사이버결제
NHN한국사이버결제
NHN
NHN벅스
